## Min/Max Heap

- Think of a min/max heap as a binary tree where the every node is smaller/larger than everything below it

- In such a structure, the root node must hold the smallest/largest value!

### Implementing Min/Max Heap

- A heap is typically represented by an array

- In the array representation of a binary tree, for a given node at index $i$
    - the node's left/right children are found at $2i+1$ and $2i+2$
        - e.g for node at index 2, the children are at index 5 and 6
    - By the same logic, a node's parent can be found at $(i-1)//2$
        - e.g. for node at index 5, the parent is at $(5-1)//2 = 2$
        - e.g. for node at index 6, the parent is at $(6-1)//2 = 2$

- A min/max heap is usually used as a priority queue of sorts, when you need to iteratively remove the object with the highest value in the queue 
    
- So what kind of operations must a priority queue (i.e. a min/max heap) support?
    - You must be able to retrieve the smallest/largest value `pop_min`
        - Getting the min/max runs in $O(1)$ time
        - But retaining the heap structure requires you to `heapify` the array after every pop, and that takes $O(\log N)$ time
    - You must be able to add an item to the queue `insert`
        - You can insert and heapify, taking $O(\log N)$ time
    - You must be able to delete an item from the queue (regardless of where it is) `delete`
        - You can delete and heapify, taking $O(\log N)$ time

In [48]:
min(None, 5)

In [57]:
import math

class MinHeap:
    def __init__(self, input_arr: list):
        self.heap_arr = self.build_heap(input_arr)
    
    def _safe_index(self, arr, index):
        if index is None:
            pass
        if 0 < index < len(arr):
            return arr[index]
        return None

    def _parent_index(self, index):
        return (index - 1)//2

    def _left_child_index(self, index):
        return 2*index + 1
    
    def _right_child_index(self, index):
        return 2*index + 2

    def build_heap(self, input_arr: list):
        tree_height = math.log2(len(input_arr))//1 + 1
        last_node_index_before_terminal_level = int(2 ** (tree_height-1) - 2)
        output_array = input_arr.copy()
        
        ## Don't need to bother about the last level, because there is no way to heapify down
        for i in range(last_node_index_before_terminal_level, -1, -1):
            output_array = self._heapify_down(output_array, i)
        return output_array

    def _heapify_up(self, arr: list, index: int):
        if arr[index] < arr[self._parent_index(index)]:
            arr[index], arr[self._parent_index(index)] = arr[self._parent_index(index)], arr[index]
            self._heapify_up(arr, self._parent_index(index))
        return arr

    def _heapify_down(self, arr: list, index: int):
        left_child_index = self._left_child_index(index)
        right_child_index = self._right_child_index(index)

        swap_with = None
        if (left_child_index < len(arr)) and (arr[index] > arr[left_child_index]):
            swap_with = left_child_index
        if (right_child_index < len(arr)) and (arr[index] > arr[right_child_index]):
            swap_with = right_child_index
        if swap_with:
            arr[index], arr[swap_with] = arr[swap_with], arr[index]
            arr = self._heapify_down(arr, swap_with)
        return arr
        
    def insert(self, val):
        self.heap_arr.append(val)
        self._heapify_up(self.heap_arr, len(self.heap_arr) - 1)

    def _delete(self, index):
        self.heap_arr[index], self.heap_arr[len(self.heap_arr)-1] = self.heap_arr[len(self.heap_arr)-1], self.heap_arr[index]
        return_val = self.heap_arr[-1]
        self.heap_arr = self.heap_arr[:-1]
        self._heapify_down(self.heap_arr, index)
        return return_val
            
    def pop_min(self):
        return self._delete(0)

    def peek(self):
        return self.heap_arr[0]

In [58]:
heap = MinHeap([1,5,2,8,9,3,10])
print(heap.heap_arr)

heap.insert(4)
print(heap.heap_arr)

heap.pop_min()
print(heap.heap_arr)

[1, 5, 2, 8, 9, 3, 10]
[1, 4, 2, 5, 9, 3, 10, 8]
[2, 4, 3, 5, 9, 8, 10]


### Heapifying

#### Insertion: Heapifying a single node

- Let's suppose we have an existing heap [1,3,4,5,6,7], all nodes respect the heap structure

- Suppose I want to add in a new node `2`

- We start by appending `2` at the end, creating the array [1,3,4,5,6,7,2]

- Now, the heap structure is broken. How can we regain the heap structure?

- Simple! We `heapify_up` the 2 until it is in the appropriate node respecting the heap structure
    - Compare 4 and 2
    - 2 < 4, so swap 2 and 4. Now the array is [1,3,2,5,6,7,4]
    - Compare 2 and 1. 2 > 1, so no swaps.
    - In the worst case, 2 gets swapped $\log N$ times (or the height of the binary tree)
    - So worst case for `heapify_up` is $O(\log N)$

#### Build Heap: Heapifying a random array

- Ok that was simply for a single node. But what if I want to do this for an entire random array?

- First thought, let's run `heapify_up` for all nodes in our array!!
    - So for each of $n$ nodes, we incur  $O(log N)$ from `heapify_up`
    - Overall time complexity becomes $O(N \log N)$

- But is there an even better way? Turns out, there is. We can `heapify_down` instead, which runs in $O(N)$ time!!

- Wait, but isn't this just doing the same thing, in the different direction? How can that possible get us from $O(N \log N)$ to $O(N)$!?!?!

- Let's check out the math:
    - **Key insight**: If you `heapify_down`, you end up doing fewer steps on average, because half of the nodes of a binary tree are in the bottom level, where you don't can't `heapify_down` any further!!
    - Why? 
        - Because $\frac{1}{2}$ of the nodes are in the bottom level of the tree, where there is no need to heapify down!
        - $\frac{1}{4}$ are in the next lower level, where it will need to move down at most 1 level
        - $\frac{1}{8}$ are in the 2nd lowest level, where it will need to move down at most 2 levels
        - etc...
        - So asymptotically, for $n$ nodes, the max number of operations is $n \cdot (\frac{1}{2} \cdot 0 + \frac{1}{4} \cdot 1 + ...) = n \cdot 1 = n$
    - Comparing what happens when you `heapify_up`
        - for $n$ nodes, the max number of operations is $n \cdot (\frac{1}{2} \cdot \log{n} + \frac{1}{4} \cdot (\log{n} - 1) + ...) \approx n \cdot \log{n}$

- $\therefore$ `heapify_down` is MUCH more efficient when building a min/max heap from a random array!

#### Why not `heapify_down` when heapifying a single node, if it's more efficient?

- When adding a new node, we always add it to the end of the array (i.e. it becomes the lowest possible node), hence we `heapify_up`

- Why not add it to the start and heapify down instead? First, let's establish that this gives a sensible answer
    - Let's take an existing heap `[3,4,6,7,9,10]`
    - Suppose, we want to add an `8` to the heap, and we append it to the array's start instead of the end
    - Array becomes `[8,3,4,6,7,9,10]`
    - Now we `heapify_down`
    - Compare `8` to `3` and `4`, and swap it with `3`
    - Array becomes `[3,8,4,6,7,9,10]`
    - Compare `8` to `6` and `7`, and swap it with `6`
    - Array becomes `[3,6,4,8,7,9,10]`

- So `heapify_down` is valid when doing insertion, but it is considered less efficient than `heapify_up` because at every step, you need to make 2 comparisons (with `curr.left` and `curr.right`) instead of 1 (with `curr.parent`)

#### Deletion: Removing a node

- When deleting a node (no matter where you are deleting from), we run the a similar `heapify_down` process
    - Swap the node you want to delete with the last node
    - Remove the last node (thereby removing the node you want to delete)
    - Run `heapify_down` on the swapped node, because it is probably larger/smaller than its children. All other nodes are not affected

- As an example, let's say we have a heap $[5, 10, 15, 20, 25, 30, 35]$
    - Let's suppose we want to delete `15`
    - Swap `15` and last node to get $[5, 10, 35, 20, 25, 30, 15]$
    - Remove last node to get $[5, 10, 35, 20, 25, 30]$
    - `heapify_down` from `35`, comparing `35` and `30`, to get $[5, 10, 30, 20, 25, 35]$
    - This is now a heap again, done in $O(\log N)$ time 


### Uses of a Heap

- Heaps are pretty useful data structures. We'll explore 3 uses:
    1. Sorting: heapsort has O(n*logn) time complexity at worst
    2. Greedy Graph Search — A* and Djikstra: greedy algorithms, such as the Djikstra’s shortest path, can use heaps to store priority-node pairs
    3. Huffman Coding: heaps can be used to store and retrieve the two lowest-frequency trees

#### HeapSort

- Heaps are one of the ways you can sort a random array
- First, build a heap from the random array, in $O(N)$
- Then, pop from the root until the array is exhausted. Each pop (deletion) runs in $O(\log N)$ time, and to exhaust the array will take us $O(N)$ time, giving us $O(N \log N)$ 

In [59]:
## Reusing MinHeap class from above

def heapsort(input_arr: list):
    heap = MinHeap(input_arr)
    sorted_arr = []
    while heap.heap_arr:
        sorted_arr.append(heap.pop_min())
    return sorted_arr

heapsort([1,5,2,8,9,3,10])

[1, 2, 3, 5, 8, 9, 10]